In [1]:
%%capture
!pip install folium matplotlib mapclassify

import pandas as pd
import geopandas as gpd
import os

FOLDER_PATH = '/content/drive/MyDrive/DSL SMART CITIES'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import NILs

In [3]:
nil = gpd.read_file(os.path.join(FOLDER_PATH, 'Processed','nil.geojson'))
nil = nil.to_crs(epsg=4326)
nil.explore()

#Import Charging Stations Data

In [4]:
cs = gpd.read_file(os.path.join(FOLDER_PATH,'COLONNINE_MILANO.txt'))
cs['numero_col'] = cs['numero_col'].astype(int)
cs = cs.drop_duplicates()
cs.columns

Index(['id_amat', 'municipio', 'id_sottonil', 'id_nil', 'nome_nil', 'cerchia',
       'ambito_id', 'ambito_nome', 'attuazione', 'tipologia', 'titolare',
       'id_via', 'nome_via', 'localita', 'numero_col', 'numero_pdr', 'infra',
       'progetto', 'note', 'ordinanza', 'anno', 'geometry'],
      dtype='object')

In [5]:
COLS_TO_KEEP = ['numero_col','geometry','anno']
cs = cs[COLS_TO_KEEP]
cs.explore(fill='anno')

In [6]:
cs

,numero_col,geometry,anno
0,2,POINT (9.19358 45.46215),2022
1,3,POINT (9.17760 45.46763),2021
2,2,POINT (9.20491 45.48448),None
3,1,POINT (9.23542 45.48428),2022
4,2,POINT (9.16917 45.43730),2022
...,...,...,...
234,1,POINT (9.19650 45.51932),2022
235,1,POINT (9.16308 45.51387),2021
236,1,POINT (9.18208 45.44674),2022
237,1,POINT (9.22713 45.45355),2022


# Space Join

In [7]:
charging_stations = nil.sjoin(cs, how='left')
charging_stations

,ID_NIL,NIL,area_sqkm,Residenti,geometry,index_right,numero_col,anno
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",104.0,1.0,2022
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",189.0,1.0,2022
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",148.0,1.0,2021
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",72.0,1.0,2021
0,1,duomo,2.341616,16608,"POLYGON ((9.19482 45.47201, 9.19473 45.47192, ...",5.0,1.0,2016
...,...,...,...,...,...,...,...,...
83,84,parco nord,1.532331,98,"POLYGON ((9.20040 45.52848, 9.20028 45.52846, ...",NaN,NaN,NaN
84,85,parco delle abbazie,13.733841,364,"POLYGON ((9.21711 45.43187, 9.21531 45.43232, ...",NaN,NaN,NaN
85,86,parco dei navigli,3.623149,354,"POLYGON ((9.13886 45.42855, 9.13769 45.42947, ...",NaN,NaN,NaN
86,87,assiano,5.840942,211,"POLYGON ((9.04687 45.46276, 9.04685 45.46278, ...",NaN,NaN,NaN


In [8]:
charging_stations = charging_stations.groupby(['ID_NIL','NIL','area_sqkm'])['numero_col'].sum().reset_index()
charging_stations = charging_stations.rename(columns={'numero_col':'charging_stations'})
charging_stations['charging_stations_per_sqkm'] = charging_stations['charging_stations']/charging_stations['area_sqkm']
charging_stations

,ID_NIL,NIL,area_sqkm,charging_stations,charging_stations_per_sqkm
0,1,duomo,2.341616,22.0,9.395223
1,10,stazione centrale - ponte seveso,1.556019,15.0,9.639987
2,11,isola,1.322887,2.0,1.511845
3,12,maciachini - maggiolina,1.674918,3.0,1.791133
4,13,greco - segnano,1.768603,2.0,1.130836
...,...,...,...,...,...
83,85,parco delle abbazie,13.733841,0.0,0.000000
84,86,parco dei navigli,3.623149,0.0,0.000000
85,87,assiano,5.840942,0.0,0.000000
86,88,parco bosco in citta',7.834006,0.0,0.000000


# OUTPUT

In [9]:
#charging_stations.to_csv(os.path.join(FOLDER_PATH,'Processed', 'open_data','charging_stations.csv'))